<a href="https://colab.research.google.com/github/hkd7148-blip/ml-portfolio/blob/main/%D0%98%D0%98_%D0%B0%D1%81%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BD%D1%82_%D0%B4%D0%BB%D1%8F_%D0%BF%D1%80%D0%B5%D1%81%D0%BA%D0%BE%D1%80%D0%B8%D0%BD%D0%B3%D0%B0_%D0%BA%D0%B0%D0%BD%D0%B4%D0%B8%D0%B4%D0%B0%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install streamlit beautifulsoup4 requests lxml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.3 MB/s eta 0:00:00


In [ ]:
%%writefile parse_hh.py
import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}

def get_html(url: str) -> str:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text

def extract_resume_data(html: str) -> str:
    # TODO: реализуем на следующем шаге
    return "RESUME_MARKDOWN_TODO"

def extract_vacancy_data(html: str) -> str:
    # TODO: реализуем на следующем шаге
    return "VACANCY_MARKDOWN_TODO"


Writing parse_hh.py


In [ ]:
!ls -la


total 20
drwxr-xr-x 1 root root 4096 Dec 12 14:26 .
drwxr-xr-x 1 root root 4096 Dec 12 14:09 ..
drwxr-xr-x 4 root root 4096 Dec  9 14:41 .config
-rw-r--r-- 1 root root  626 Dec 12 14:26 parse_hh.py
drwxr-xr-x 1 root root 4096 Dec  9 14:42 sample_data


In [ ]:
%%writefile parse_hh.py
import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    )
}

def get_html(url: str) -> str:
    response = requests.get(url, headers=HEADERS, timeout=30)
    response.raise_for_status()
    return response.text


def extract_resume_data(html: str) -> str:
    """
    Парсинг резюме HH.ru
    Возвращает структурированный текст
    """
    soup = BeautifulSoup(html, "lxml")

    content = soup.find("div", {"data-qa": "resume-block-container"})
    if not content:
        return "Резюме не найдено"

    text_parts = []

    # Заголовок
    title = soup.find("h1")
    if title:
        text_parts.append(f"ДОЛЖНОСТЬ: {title.get_text(strip=True)}")

    # Опыт работы
    experience_blocks = soup.find_all("div", {"data-qa": "resume-block-experience"})
    for exp in experience_blocks:
        text_parts.append("\nОПЫТ РАБОТЫ:")
        for item in exp.find_all("div", recursive=False):
            text_parts.append(item.get_text(" ", strip=True))

    # Навыки
    skills = soup.find_all("span", {"data-qa": "resume-skill-name"})
    if skills:
        text_parts.append("\nНАВЫКИ:")
        for skill in skills:
            text_parts.append(f"- {skill.get_text(strip=True)}")

    return "\n".join(text_parts)


def extract_vacancy_data(html: str) -> str:
    """
    Парсинг вакансии HH.ru
    Возвращает структурированный текст
    """
    soup = BeautifulSoup(html, "lxml")

    text_parts = []

    title = soup.find("h1")
    if title:
        text_parts.append(f"ВАКАНСИЯ: {title.get_text(strip=True)}")

    description = soup.find("div", {"data-qa": "vacancy-description"})
    if description:
        text_parts.append("\nОПИСАНИЕ:")
        text_parts.append(description.get_text("\n", strip=True))

    return "\n".join(text_parts)


Overwriting parse_hh.py


In [ ]:
from parse_hh import extract_resume_data, extract_vacancy_data

test_html = """
<html>
<h1>Python Developer</h1>
<div data-qa="resume-block-container">
  <div data-qa="resume-block-experience">
    <div>Разработка backend-сервисов</div>
    <div>Работа с API</div>
  </div>
  <span data-qa="resume-skill-name">Python</span>
  <span data-qa="resume-skill-name">SQL</span>
</div>
</html>
"""

print(extract_resume_data(test_html))


ДОЛЖНОСТЬ: Python Developer

ОПЫТ РАБОТЫ:
Разработка backend-сервисов
Работа с API

НАВЫКИ:
- Python
- SQL


In [ ]:
%%writefile scoring.py
import re

def extract_skills(text: str) -> set:
    """
    Простейшее извлечение навыков из текста
    """
    skills = set()
    for line in text.splitlines():
        line = line.lower()
        if line.startswith("- "):
            skills.add(line.replace("- ", "").strip())
    return skills


def skill_match_score(vacancy_text: str, resume_text: str) -> float:
    vacancy_skills = extract_skills(vacancy_text)
    resume_skills = extract_skills(resume_text)

    if not vacancy_skills:
        return 0.0

    matched = vacancy_skills & resume_skills
    return len(matched) / len(vacancy_skills)


def resume_quality_score(resume_text: str) -> float:
    """
    Оценка качества резюме
    """
    score = 0

    if len(resume_text) > 300:
        score += 0.3

    if re.search(r"\d+%", resume_text):
        score += 0.3

    if "опыт" in resume_text.lower():
        score += 0.2

    if "результ" in resume_text.lower():
        score += 0.2

    return min(score, 1.0)


def prescore(vacancy_text: str, resume_text: str) -> dict:
    skill_score = skill_match_score(vacancy_text, resume_text)
    quality_score = resume_quality_score(resume_text)

    final_score = round((skill_score * 0.6 + quality_score * 0.4) * 10, 1)

    return {
        "skill_match": round(skill_score * 10, 1),
        "resume_quality": round(quality_score * 10, 1),
        "final_score": final_score
    }


Writing scoring.py


In [ ]:
from scoring import prescore

vacancy_text = """
ТРЕБОВАНИЯ:
- Python
- SQL
- API
"""

resume_text = """
ДОЛЖНОСТЬ: Python Developer

ОПЫТ РАБОТЫ:
Разработка backend-сервисов
Работа с API

НАВЫКИ:
- Python
- SQL
"""

result = prescore(vacancy_text, resume_text)
print(result)


{'skill_match': 6.7, 'resume_quality': 2.0, 'final_score': 4.8}


In [ ]:
%%writefile ai_assistant.py
def build_prescoring_prompt(vacancy_text: str, resume_text: str) -> str:
    """
    Формирует промпт для ИИ-ассистента прескоринга кандидатов
    """

    prompt = f"""
Ты — ИИ-ассистент для прескоринга кандидатов на вакансии.

Твоя задача — оценить, насколько кандидат соответствует вакансии,
на основе текста вакансии и текста резюме.

Работай строго по этапам.

=== ТЕКСТ ВАКАНСИИ ===
{vacancy_text}

=== ТЕКСТ РЕЗЮМЕ ===
{resume_text}

=== ЭТАП 1. АНАЛИТИЧЕСКИЙ КОММЕНТАРИЙ ===
Составь краткий аналитический комментарий (5–7 предложений), в котором:
- объясни степень соответствия кандидата вакансии
- укажи ключевые совпадения и расхождения
- отметь недостающие навыки или опыт

=== ЭТАП 2. ОЦЕНКА КАЧЕСТВА РЕЗЮМЕ ===
Оцени качество резюме по шкале от 1 до 10, учитывая:
- понятность описания задач и проблем
- описание способов их решения
- наличие результатов и достижений
- структуру и аккуратность оформления
- способность кандидата анализировать свой вклад в бизнес

После оценки дай краткое пояснение.

=== ЭТАП 3. ИТОГОВАЯ ОЦЕНКА ===
Выставь итоговую оценку соответствия кандидата вакансии по шкале от 1 до 10.
Эта оценка должна учитывать в том числе качество резюме.

=== ФОРМАТ ОТВЕТА ===
Аналитический комментарий:
<текст>

Качество резюме: X/10
Пояснение:
<текст>

Итоговая оценка соответствия: Y/10
Пояснение:
<текст>
"""
    return prompt


Writing ai_assistant.py


In [ ]:
%%writefile app.py
import streamlit as st
from parse_hh import get_html, extract_resume_data, extract_vacancy_data
from scoring import prescore
from ai_assistant import build_prescoring_prompt

st.set_page_config(page_title="ИИ-прескоринг кандидатов", layout="centered")

st.title("ИИ-ассистент для прескоринга кандидатов")

vacancy_url = st.text_input("Ссылка на вакансию HH")
resume_url = st.text_input("Ссылка на резюме HH")

if st.button("Оценить кандидата"):
    if not vacancy_url or not resume_url:
        st.error("Введите обе ссылки")
    else:
        vacancy_html = get_html(vacancy_url)
        resume_html = get_html(resume_url)

        vacancy_text = extract_vacancy_data(vacancy_html)
        resume_text = extract_resume_data(resume_html)

        numeric_score = prescore(vacancy_text, resume_text)
        ai_prompt = build_prescoring_prompt(vacancy_text, resume_text)

        st.subheader("Числовой прескоринг (алгоритм)")
        st.json(numeric_score)

        st.subheader("Промпт ИИ-ассистента")
        st.text_area("Используемый промпт", ai_prompt, height=400)


Overwriting app.py


In [ ]:
!pip install -q streamlit pyngrok


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.58.240.21:8501

y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏your url is: https://slick-nails-chew.loca.lt
y
  Stopping...
^C


In [ ]:
!curl https://loca.lt/mytunnelpassword


34.58.240.21